In [1]:
import krakenex
import pandas as pd
from pykrakenapi import KrakenAPI

In [2]:
def fatching_data(asset_pair, interval):
    api = krakenex.API()
    k = KrakenAPI(api)
    ohlc, last = k.get_ohlc_data(asset_pair, interval=interval)
    ohlc.reset_index(drop=True, inplace=True)
    ohlc.drop('time', axis=1, inplace=True)
    return(ohlc)

In [3]:
candle_rankings = {
        "CDL3LINESTRIKE_Bull": 1,
        "CDL3LINESTRIKE_Bear": 2,
        "CDL3BLACKCROWS_Bull": 3,
        "CDL3BLACKCROWS_Bear": 3,
        "CDLEVENINGSTAR_Bull": 4,
        "CDLEVENINGSTAR_Bear": 4,
        "CDLTASUKIGAP_Bull": 5,
        "CDLTASUKIGAP_Bear": 5,
        "CDLINVERTEDHAMMER_Bull": 6,
        "CDLINVERTEDHAMMER_Bear": 6,
        "CDLMATCHINGLOW_Bull": 7,
        "CDLMATCHINGLOW_Bear": 7,
        "CDLABANDONEDBABY_Bull": 8,
        "CDLABANDONEDBABY_Bear": 8,
        "CDLBREAKAWAY_Bull": 10,
        "CDLBREAKAWAY_Bear": 10,
        "CDLMORNINGSTAR_Bull": 12,
        "CDLMORNINGSTAR_Bear": 12,
        "CDLPIERCING_Bull": 13,
        "CDLPIERCING_Bear": 13,
        "CDLSTICKSANDWICH_Bull": 14,
        "CDLSTICKSANDWICH_Bear": 14,
        "CDLTHRUSTING_Bull": 15,
        "CDLTHRUSTING_Bear": 15,
        "CDLINNECK_Bull": 17,
        "CDLINNECK_Bear": 17,
        "CDL3INSIDE_Bull": 20,
        "CDL3INSIDE_Bear": 56,
        "CDLHOMINGPIGEON_Bull": 21,
        "CDLHOMINGPIGEON_Bear": 21,
        "CDLDARKCLOUDCOVER_Bull": 22,
        "CDLDARKCLOUDCOVER_Bear": 22,
        "CDLIDENTICAL3CROWS_Bull": 24,
        "CDLIDENTICAL3CROWS_Bear": 24,
        "CDLMORNINGDOJISTAR_Bull": 25,
        "CDLMORNINGDOJISTAR_Bear": 25,
        "CDLXSIDEGAP3METHODS_Bull": 27,
        "CDLXSIDEGAP3METHODS_Bear": 26,
        "CDLTRISTAR_Bull": 28,
        "CDLTRISTAR_Bear": 76,
        "CDLGAPSIDESIDEWHITE_Bull": 46,
        "CDLGAPSIDESIDEWHITE_Bear": 29,
        "CDLEVENINGDOJISTAR_Bull": 30,
        "CDLEVENINGDOJISTAR_Bear": 30,
        "CDL3WHITESOLDIERS_Bull": 32,
        "CDL3WHITESOLDIERS_Bear": 32,
        "CDLONNECK_Bull": 33,
        "CDLONNECK_Bear": 33,
        "CDL3OUTSIDE_Bull": 34,
        "CDL3OUTSIDE_Bear": 39,
        "CDLRICKSHAWMAN_Bull": 35,
        "CDLRICKSHAWMAN_Bear": 35,
        "CDLSEPARATINGLINES_Bull": 36,
        "CDLSEPARATINGLINES_Bear": 40,
        "CDLLONGLEGGEDDOJI_Bull": 37,
        "CDLLONGLEGGEDDOJI_Bear": 37,
        "CDLHARAMI_Bull": 38,
        "CDLHARAMI_Bear": 72,
        "CDLLADDERBOTTOM_Bull": 41,
        "CDLLADDERBOTTOM_Bear": 41,
        "CDLCLOSINGMARUBOZU_Bull": 70,
        "CDLCLOSINGMARUBOZU_Bear": 43,
        "CDLTAKURI_Bull": 47,
        "CDLTAKURI_Bear": 47,
        "CDLDOJISTAR_Bull": 49,
        "CDLDOJISTAR_Bear": 51,
        "CDLHARAMICROSS_Bull": 50,
        "CDLHARAMICROSS_Bear": 80,
        "CDLADVANCEBLOCK_Bull": 54,
        "CDLADVANCEBLOCK_Bear": 54,
        "CDLSHOOTINGSTAR_Bull": 55,
        "CDLSHOOTINGSTAR_Bear": 55,
        "CDLMARUBOZU_Bull": 71,
        "CDLMARUBOZU_Bear": 57,
        "CDLUNIQUE3RIVER_Bull": 60,
        "CDLUNIQUE3RIVER_Bear": 60,
        "CDL2CROWS_Bull": 61,
        "CDL2CROWS_Bear": 61,
        "CDLBELTHOLD_Bull": 62,
        "CDLBELTHOLD_Bear": 63,
        "CDLHAMMER_Bull": 65,
        "CDLHAMMER_Bear": 65,
        "CDLHIGHWAVE_Bull": 67,
        "CDLHIGHWAVE_Bear": 67,
        "CDLSPINNINGTOP_Bull": 69,
        "CDLSPINNINGTOP_Bear": 73,
        "CDLUPSIDEGAP2CROWS_Bull": 74,
        "CDLUPSIDEGAP2CROWS_Bear": 74,
        "CDLGRAVESTONEDOJI_Bull": 77,
        "CDLGRAVESTONEDOJI_Bear": 77,
        "CDLHIKKAKEMOD_Bull": 82,
        "CDLHIKKAKEMOD_Bear": 81,
        "CDLHIKKAKE_Bull": 85,
        "CDLHIKKAKE_Bear": 83,
        "CDLENGULFING_Bull": 84,
        "CDLENGULFING_Bear": 91,
        "CDLMATHOLD_Bull": 86,
        "CDLMATHOLD_Bear": 86,
        "CDLHANGINGMAN_Bull": 87,
        "CDLHANGINGMAN_Bear": 87,
        "CDLRISEFALL3METHODS_Bull": 94,
        "CDLRISEFALL3METHODS_Bear": 89,
        "CDLKICKING_Bull": 96,
        "CDLKICKING_Bear": 102,
        "CDLDRAGONFLYDOJI_Bull": 98,
        "CDLDRAGONFLYDOJI_Bear": 98,
        "CDLCONCEALBABYSWALL_Bull": 101,
        "CDLCONCEALBABYSWALL_Bear": 101,
        "CDL3STARSINSOUTH_Bull": 103,
        "CDL3STARSINSOUTH_Bear": 103,
        "CDLDOJI_Bull": 104,
        "CDLDOJI_Bear": 104
    }

In [4]:
import numpy as np
import talib
from itertools import compress

def recognize_candlestick(df):
    """
    Recognizes candlestick patterns and appends 2 additional columns to df;
    1st - Best Performance candlestick pattern matched by www.thepatternsite.com
    2nd - # of matched patterns
    """

    op = df['open'].astype(float)
    hi = df['high'].astype(float)
    lo = df['low'].astype(float)
    cl = df['close'].astype(float)

    candle_names = talib.get_function_groups()['Pattern Recognition']

    # patterns not found in the patternsite.com
    exclude_items = ('CDLCOUNTERATTACK',
                     'CDLLONGLINE',
                     'CDLSHORTLINE',
                     'CDLSTALLEDPATTERN',
                     'CDLKICKINGBYLENGTH')

    candle_names = [candle for candle in candle_names if candle not in exclude_items]


    # create columns for each candle
    for candle in candle_names:
        # below is same as;
        # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
        df[candle] = getattr(talib, candle)(op, hi, lo, cl)


    df['candlestick_pattern'] = np.nan
    df['candlestick_match_count'] = np.nan
    for index, row in df.iterrows():

        # no pattern found
        if len(row[candle_names]) - sum(row[candle_names] == 0) == 0:
            df.loc[index,'candlestick_pattern'] = "NO_PATTERN"
            df.loc[index, 'candlestick_match_count'] = 0
        # single pattern found
        elif len(row[candle_names]) - sum(row[candle_names] == 0) == 1:
            # bull pattern 100 or 200
            if any(row[candle_names].values > 0):
                pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
                df.loc[index, 'candlestick_pattern'] = pattern
                df.loc[index, 'candlestick_match_count'] = 1
            # bear pattern -100 or -200
            else:
                pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
                df.loc[index, 'candlestick_pattern'] = pattern
                df.loc[index, 'candlestick_match_count'] = 1
        # multiple patterns matched -- select best performance
        else:
            # filter out pattern names from bool list of values
            patterns = list(compress(row[candle_names].keys(), row[candle_names].values != 0))
            container = []
            for pattern in patterns:
                if row[pattern] > 0:
                    container.append(pattern + '_Bull')
                else:
                    container.append(pattern + '_Bear')
            rank_list = [candle_rankings[p] for p in container]
            if len(rank_list) == len(container):
                rank_index_best = rank_list.index(min(rank_list))
                df.loc[index, 'candlestick_pattern'] = container[rank_index_best]
                df.loc[index, 'candlestick_match_count'] = len(container)
    # clean up candle columns
    cols_to_drop = candle_names + list(exclude_items)
    df.to_csv("Stock_Candlestic_Pattern.csv", index=False)
#     df.drop(cols_to_drop, axis = 1, inplace = True)

    return df

In [5]:
# from plotly.offline import plot
# import plotly.graph_objs as go

# o = df['open'].astype(float)
# h = df['high'].astype(float)
# l = df['low'].astype(float)
# c = df['close'].astype(float)

# trace = go.Candlestick(
#             open=o,
#             high=h,
#             low=l,
#             close=c)
# data = [trace]

# ##plot(data, filename='go_candle1.html')


# layout = {
#     'title': 'Bitcoin Candlestick Chart',
#     'yaxis': {'title': 'Price'},
#     'xaxis': {'title': 'Index Number'},

# }
# fig = dict(data=data, layout=layout)
# plot(fig, filename='btc_candles')

In [6]:
def remove_unnecessary_feature(df):
    df.drop(["candlestick_pattern", "time"], axis=1, inplace=True)
    return df


In [7]:
def remove_Na_values(df):
    if (df.isna().sum().sum() > 0):
        df.fillna(value=0, inplace=True,)
    else:
        print("Not Found any Null values")
    return df

In [8]:
def remove_correlated_feature(df):
    correlated_features = set()
    correlation_matrix = df.corr()
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.8:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
    correlated_features.discard("close")
    df.drop(labels=correlated_features, axis=1, inplace=True)
    return df

In [9]:
def preprocessing(df):
#     df = remove_unnecessary_feature(df)
    df = remove_Na_values(df)
    df = remove_correlated_feature(df)
    df.to_csv("Cleaned_Data.csv", index=False)
    return df

In [10]:
def create_data(df,assesst_name, window=30, ):
    """ This function will create the input(sequence) and output(status) data

    *************parameter**************
    df : Dataframe of sequential data
    window : (int) any int values defines the window size for look back
             By default window is 30

    ************Return******************
    Return a New Dataframe
    Dataframe["InputSequence"] must be a 2-D array with dimension of window and feaure of Dataframe
    0-> bearish
    1-> bullish

    """
    target = []
    sequence = []
    trend = []
    Assessts = []
    for i in range(window , len(df) - 15):
        ## append new timestamp in sequence
        value = df.loc[i - window:i-1].values
        sequence.append(value)
        Assessts.append(assesst_name)

        ## Check the trend in input data
        if (value[0, 4] > value[-1, 4]):
            trend.append("Down_Trend")
        elif (value[0, 4] < value[-1, 4]):
            trend.append("Up_Trend")
        else:
            trend.append("Flat")

        ## updtion the status of respective timestamps
        if (df.iloc[i + 15].close > df.iloc[i].close):
            target.append(1)
        else:
            target.append(0)

    #     target = np.array(status)
    #     sequence = np.array(sequence)
    print(len(sequence))
    data_df = pd.DataFrame(sequence, columns=["InputSequence"], )
    data_df["Trend"] = trend
    data_df["Target"] = target
    data_df["Assessts"] = Assessts
    # data_df.to_csv("Generated_data_with_TargetAndTrend.csv",index=False)
    return data_df

In [11]:
def fatch_to_final_data(assest_pair, interval =1):
    if isinstance(assest_pair, list):
        final_df = pd.DataFrame(columns=['InputSequence', 'Trend', 'Target', 'Assessts'])

        for assesst in assest_pair:
            ohlc_data = fatching_data(asset_pair=assesst, interval=interval)
            candlestic_pattern_data = recognize_candlestick(ohlc_data)
            preprocess_data = preprocessing(candlestic_pattern_data)
            target_data = create_data(preprocess_data, assesst_name=assesst)

            final_df = final_df.append(target_data, ignore_index=True,)
        final_df.to_csv("Generated_data_with_TargetAndTrend.csv",index=False)
        print("Generated Data has been saved as 'Generated_data_with_TargetAndTrend.csv' !")
        return final_df
    else:
        return("Given assest_pair is not List !")

# if __name__ == "__main__":
    # assest_name = input("Enter Assets pair Keyword : ")
#     data = fatch_to_final_data(["BCHUSD", "BATXBT"])

#     print(data.shape)

In [12]:
df=fatch_to_final_data(["BCHUSD", "BATXBT"])

Not Found any Null values
675
Not Found any Null values
497
Generated Data has been saved as 'Generated_data_with_TargetAndTrend.csv' !


In [13]:
df

,InputSequence,Trend,Target,Assessts
0,"[[230.0, 230.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Flat,1,BCHUSD
1,"[[230.0, 230.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Flat,1,BCHUSD
2,"[[230.0, 230.0, 230.0, 0.00046064, 3, 0, 0, 0,...",Down_Trend,1,BCHUSD
3,"[[230.0, 230.0, 230.0, 1.0, 1, 0, 0, 0, 0, 0, ...",Down_Trend,0,BCHUSD
4,"[[230.0, 230.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Flat,0,BCHUSD
...,...,...,...,...
1167,"[[2.864e-05, 2.864e-05, 0.0, 0.0, 0, 0, 0, 0, ...",Flat,0,BATXBT
1168,"[[2.864e-05, 2.864e-05, 0.0, 0.0, 0, 0, 0, 0, ...",Flat,0,BATXBT
1169,"[[2.864e-05, 2.864e-05, 0.0, 0.0, 0, 0, 0, 0, ...",Flat,0,BATXBT
1170,"[[2.864e-05, 2.864e-05, 0.0, 0.0, 0, 0, 0, 0, ...",Flat,0,BATXBT


In [14]:
df['InputSequence'][0]

array([[230.0, 230.0, 0.0, ..., 0, 'NO_PATTERN', 0.0],
       [230.0, 230.0, 0.0, ..., 0, 'NO_PATTERN', 0.0],
       [230.0, 230.0, 230.0, ..., 0, 'NO_PATTERN', 0.0],
       ...,
       [230.2, 230.2, 0.0, ..., 0, 'CDLDOJI_Bull', 1.0],
       [230.4, 230.2, 230.2, ..., 0, 'NO_PATTERN', 0.0],
       [230.4, 230.4, 0.0, ..., 0, 'CDLDOJI_Bull', 1.0]], dtype=object)

In [15]:
df_uptrend = df[df.Trend == 'Up_Trend']

In [16]:
df_uptrend

,InputSequence,Trend,Target,Assessts
6,"[[230.0, 230.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,0,BCHUSD
9,"[[230.0, 230.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,1,BCHUSD
18,"[[230.2, 230.2, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,1,BCHUSD
21,"[[230.2, 230.2, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,0,BCHUSD
22,"[[230.2, 230.2, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,0,BCHUSD
...,...,...,...,...
660,"[[229.6, 229.6, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,0,BCHUSD
665,"[[230.5, 230.5, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,0,BCHUSD
668,"[[230.5, 230.5, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,0,BCHUSD
669,"[[230.5, 230.5, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",Up_Trend,0,BCHUSD


In [17]:
df_uptrend.shape

(191, 4)

In [18]:
df_flat = df[df.Trend == 'Flat']

In [19]:
df_flat.shape

(798, 4)

In [20]:
df_Down_Trend = df[df.Trend == 'Down_Trend']

In [21]:
df_Down_Trend.shape

(183, 4)

In [22]:
x_uptrend = df_uptrend.iloc[:,0].to_list()

In [23]:
# pd.DataFrame(x,)
for i in range(len(x_uptrend)):
    x_uptrend[i]=x_uptrend[i].ravel()
    

In [24]:
x_uptrend[0].shape

(1800,)

In [25]:
x_uptrend=pd.DataFrame(x_uptrend)

In [26]:
x_uptrend['Target']=df_uptrend['Target'].to_list()

In [46]:
x_uptrend = x_uptrend.drop([1798],axis=1)

In [47]:
x_uptrend

,0,1,2,3,4,5,6,7,8,9,...,1790,1791,1792,1793,1794,1795,1796,1797,1799,Target
0,230.0,230.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.0,0
1,230.0,230.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,1
2,230.2,230.2,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,1
3,230.2,230.2,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0
4,230.2,230.2,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,229.6,229.6,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0
187,230.5,230.5,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0
188,230.5,230.5,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
189,230.5,230.5,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.0,0


In [ ]:
# applying Svm

In [28]:
x_downtrend = df_Down_Trend.iloc[:,0].to_list()

In [29]:
for i in range(len(x_downtrend)):
    x_downtrend[i]=x_downtrend[i].ravel()

In [30]:
x_downtrend[0].shape

(1800,)

In [31]:
x_downtrend=pd.DataFrame(x_downtrend)

In [32]:
x_downtrend['Target']=df_Down_Trend['Target'].to_list()

In [33]:
x_downtrend

,0,1,2,3,4,5,6,7,8,9,...,1791,1792,1793,1794,1795,1796,1797,1798,1799,Target
0,230.0,230.0,230.0,0.000461,3,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,1
1,230.0,230.0,230.0,1.000000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
2,230.0,230.0,230.0,3.755000,2,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,1
3,230.2,230.2,230.2,1.006342,1,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
4,230.4,230.2,230.2,15.000000,8,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLBELTHOLD_Bull,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,229.6,229.7,229.4,15.000608,4,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
179,230.0,229.6,229.6,0.000208,1,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
180,230.5,229.8,229.8,0.000384,6,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLGAPSIDESIDEWHITE_Bear,2.0,0
181,230.6,230.5,230.5,13.474011,4,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0


In [34]:
x_flat = df_flat.iloc[:,0].to_list()

In [35]:
for i in range(len(x_flat)):
    x_flat[i]=x_flat[i].ravel()

In [36]:
x_flat[0].shape

(1800,)

In [37]:
x_flat=pd.DataFrame(x_flat)

In [38]:
x_flat['Target']=df_flat['Target'].to_list()

In [39]:
x_flat

,0,1,2,3,4,5,6,7,8,9,...,1791,1792,1793,1794,1795,1796,1797,1798,1799,Target
0,230.000000,230.000000,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,1
1,230.000000,230.000000,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,1
2,230.000000,230.000000,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
3,230.000000,230.000000,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
4,230.000000,230.000000,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,0.000029,0.000029,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
794,0.000029,0.000029,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
795,0.000029,0.000029,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0
796,0.000029,0.000029,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,CDLDOJI_Bull,1.0,0


In [ ]:
# Applying Svm